<a href="https://colab.research.google.com/github/brenoskuk/recvis/blob/main/SinGan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SinGan project

## MVA RecVis

### Collaborators: Breno Skuk and Claudia Torres Pérez

In [ ]:
import numpy as np

In [1]:
!git clone https://github.com/tamarott/SinGAN.git

Cloning into 'SinGAN'...
remote: Enumerating objects: 790, done.
remote: Total 790 (delta 0), reused 0 (delta 0), pack-reused 790
Receiving objects: 100% (790/790), 94.89 MiB | 41.45 MiB/s, done.
Resolving deltas: 100% (304/304), done.
